In [14]:
import sparknlp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pyspark.sql import SparkSession
from pyspark.sql import Row
# from pyspark.sql.types import StringType, StructType, StructField
from pyspark.sql.types import *
# import pyspark.sql.functions as F
from pyspark.sql.functions import *

import warnings
warnings.filterwarnings('ignore')

spark = sparknlp.start()

In [15]:
spark = SparkSession.builder \
    .appName("Spark NLP") \
    .master("local[*]") \
    .config("spark.driver.memory", "16G") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryoserializer.buffer.max", "2000M") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.3.3") \
    .getOrCreate()

In [16]:
import transformers

MODEL_NAME = "flan-t5-small"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

In [17]:
!optimum-cli export onnx --task text2text-generation-with-past --model {MODEL_NAME} {EXPORT_PATH}

/root/anaconda3/envs/sp/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/root/anaconda3/envs/sp/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Framework not specified. Using pt to export the model.
Using the export variant default. Available variants are:
    - default: The default ONNX variant.

***** Exporting submodel 1/3: T5Stack *****
Using framework PyTorch: 2.3.0+cpu
Overriding 1 configuration item(s)
	- use_cache -> False

***** Exporting submodel 2/3: T5ForConditionalGeneration *****
Using framework PyTorch: 2.3.0+cpu
Overriding 1 configuration item(s)
	- use_cache -> True
/root/anaconda3/envs/sp/lib/python3.11

In [18]:
# !ls -l {EXPORT_PATH}

! mkdir -p {EXPORT_PATH}/assets
! mv -t {EXPORT_PATH}/assets {EXPORT_PATH}/assets/spiece.model

!ls -l {EXPORT_PATH}/assets

mv: cannot stat 'onnx_models/flan-t5-small/assets/spiece.model': No such file or directory
total 0


In [19]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 5.3.3
setup Colab for PySpark 3.2.3 and Spark NLP 5.3.3


In [20]:
import sparknlp

# let's start Spark with Spark NLP
spark = sparknlp.start()

In [21]:
from sparknlp.annotator import *

T5 = T5Transformer.loadSavedModel(EXPORT_PATH, spark)\
  .setUseCache(True) \
  .setTask("summarize:") \
  .setMaxOutputLength(200)

Py4JJavaError: An error occurred while calling z:com.johnsnowlabs.nlp.annotators.seq2seq.T5Transformer.loadSavedModel.
: java.lang.NoClassDefFoundError: com/amazonaws/AmazonServiceException
	at com.johnsnowlabs.ml.util.LoadExternalModel$.modelSanityCheck(LoadExternalModel.scala:137)
	at com.johnsnowlabs.nlp.annotators.seq2seq.ReadT5TransformerDLModel.loadSavedModel(T5Transformer.scala:635)
	at com.johnsnowlabs.nlp.annotators.seq2seq.ReadT5TransformerDLModel.loadSavedModel$(T5Transformer.scala:633)
	at com.johnsnowlabs.nlp.annotators.seq2seq.T5Transformer$.loadSavedModel(T5Transformer.scala:694)
	at com.johnsnowlabs.nlp.annotators.seq2seq.T5Transformer.loadSavedModel(T5Transformer.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflec

In [ ]:
T5.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

NameError: name 'T5' is not defined

In [ ]:
!rm -rf {EXPORT_PATH}